In [1]:
import os
import pandas as pd
import json

project_dir = os.path.join(os.curdir, "project")
project_dir

'.\\project'

In [2]:
training_result = {}
inf_result = {}
sorted_inf_result = {}

for exp in os.listdir(project_dir):
    directory = os.path.join(project_dir, exp)

    try:
        file = open(os.path.join(directory, "mlperf_cloudmask.log"), "r")
    except FileNotFoundError:
        continue

    data = None
    for line in file:
        if "result" in line:
            data = line.rstrip()

    if not data:
        continue

    data = json.loads(data[9:])

    loss = data["value"]["training"]["history"]["loss"]
    accuracy = data["value"]["training"]["history"]["accuracy"]
    val_loss = data["value"]["training"]["history"]["val_loss"]
    val_accuracy = data["value"]["training"]["history"]["val_accuracy"]
    epochs = len(accuracy)

    df = pd.DataFrame(list(zip(loss, accuracy, val_loss, val_accuracy)), columns=["Loss", "Accuracy", "ValLoss", "ValAccuracy"])

    training_result[epochs] = df

    inference = data["value"]["inference"]["accuracy"]
    if len(inference) != 100:
        continue
    inf_result[epochs] = inference

    for i in range(len(inference)):
        inference[i] = float(inference[i])

    inference.sort(reverse=True)
    sorted_inf_result[epochs] = inference


In [3]:
from pprint import pprint
inf_df = pd.DataFrame(inf_result)
sorted_inf_df = pd.DataFrame(sorted_inf_result)

#pprint(sorted_inf_result)
display(sorted_inf_df)

,1,3,50
0,0.962921,0.962921,0.962921
1,0.960064,0.960064,0.960064
2,0.959243,0.959243,0.959243
3,0.956681,0.956681,0.956681
4,0.946067,0.946067,0.946067
...,...,...,...
95,0.580571,0.580571,0.580571
96,0.553338,0.553338,0.553338
97,0.531188,0.531188,0.531188
98,0.503403,0.503403,0.503403
